In [1]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
# old data set 
df = pd.read_csv("../all_data_Char.csv")

In [15]:
# create the test and the train data set. => old data set
disordered = df.loc[df["label"]==1.0]
ordered = df.loc[df["label"]==0.0]
# select indices based on a random distributions 
disordered["is_train"] = np.random.uniform(0, 1, len(disordered)) <= 0.885 # calculated based on test/total
ordered["is_train"] = np.random.uniform(0,1, len(ordered)) <= 0.9947 # calculated based on test/total

/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sequence,seq_len,molecular_weight,hydrophobicity,instability_index,aromaticity,label,is_train
0,MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKR...,529,59138.7350,-0.611153,58.945936,0.058601,1.0,True
1,MKTQRDGHSLGRWSLVLLLLGLVMPLAIIAQVLSYKEAVLRAIDGI...,170,19301.1649,-0.441765,37.586471,0.064706,1.0,True
2,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...,107,12298.1162,-0.919626,46.321495,0.037383,1.0,True
3,MGDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPG...,105,11832.6058,-0.875238,15.507619,0.085714,1.0,True
4,MPKRGKKGAVAEDGDELRTEPEAKKSKTAAKKNDKEAAGEGPALYE...,318,35554.0730,-0.583019,43.411950,0.088050,1.0,True


In [46]:
# old data set 
df2 = pd.concat([disordered,ordered])
df2 = df2.drop(columns=["sequence"])
train, test = df2[df2['is_train']==True], df2[df2['is_train']==False]

In [3]:
df1 = pd.read_csv("../all_data_1.csv")
df2 = pd.read_csv("../all_data_2.csv")
df = pd.concat([df1,df2])
disordered = df.loc[df["label"]==1.0]
label_dis = disordered["label"]
disordered = disordered.drop(columns=["label", "sequence"])
ordered = df.loc[df["label"]==0.0]
label_ord = ordered["label"]
ordered = ordered.drop(columns=["label", "sequence"])

# 16316 disordered => 0.236 
# 240481 total ordered 
# 7704 => 3852 
#split ordered
xo_train, xo_test, yo_train, yo_test = train_test_split(disordered, label_dis, test_size=0.236, random_state=42)
#split disordered
xd_train, xd_test, yd_train, yd_test = train_test_split(ordered, label_ord, test_size=0.016, random_state=42)
#concatenate the two
train_features = pd.concat([xo_train, xd_train]).to_numpy().astype('float32')
train_labels = pd.concat([yo_train, yd_train]).to_numpy().astype('float32')

test_features = pd.concat([xo_test, xd_test]).to_numpy().astype('float32')
test_labels = pd.concat([yo_test, yd_test]).to_numpy().astype('float32')

print('Number of observations in the training data:', len(train_features))
print('Number of observations in the test data:',len(test_features))

Number of observations in the training data: 249098
Number of observations in the test data: 7699


In [50]:
#features = df2.columns[1:5]
#y = train.label.to_list()

In [10]:
clf = RandomForestClassifier(n_jobs=2, n_estimators =25, random_state=0)
clf.fit(train_features, train_labels)
clf.predict(test_features)
clf.predict_proba(test_features)[0:10]

array([[0.96      , 0.04      ],
       [0.36190476, 0.63809524],
       [0.12      , 0.88      ],
       [0.96      , 0.04      ],
       [1.        , 0.        ],
       [0.83217313, 0.16782687],
       [0.96      , 0.04      ],
       [0.24      , 0.76      ],
       [0.72589346, 0.27410654],
       [0.965722  , 0.034278  ]])

In [11]:
preds = clf.predict(test_features)

In [12]:
pd.crosstab(test_labels, preds, rownames=['Actual label'], colnames=['Predicted label'])

Predicted label,0.0,1.0
Actual label,,
0.0,3834,14
1.0,1956,1895


In [13]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(test_labels,preds))
print(classification_report(test_labels,preds))

[[3834   14]
 [1956 1895]]
              precision    recall  f1-score   support

         0.0       0.66      1.00      0.80      3848
         1.0       0.99      0.49      0.66      3851

    accuracy                           0.74      7699
   macro avg       0.83      0.74      0.73      7699
weighted avg       0.83      0.74      0.73      7699



In [8]:
list(zip(train_features, clf.feature_importances_))

[(array([ 1.2400000e+02,  1.3690179e+04, -6.6290325e-01,  4.2120968e+01,
          7.2580643e-02,  9.6774191e-02,  6.4516127e-02,  4.0322579e-02,
          4.0322579e-02,  2.4193548e-02,  2.4193548e-02,  3.2258064e-02,
          2.4193548e-02,  8.0645159e-02,  1.6129032e-02,  3.2258064e-02,
          8.0645159e-02,  3.2258064e-02,  5.6451611e-02,  3.2258064e-02,
          1.2096774e-01,  8.0645159e-02,  7.2580643e-02,  0.0000000e+00,
          4.8387095e-02], dtype=float32),
  0.04399511860248234),
 (array([ 1.5000000e+02,  1.7455055e+04, -3.1000000e-01,  4.1368668e+01,
          1.0000000e-01,  3.3333335e-02,  9.3333334e-02,  8.6666666e-02,
          5.9999999e-02,  2.6666667e-02,  7.3333330e-02,  2.0000000e-02,
          5.9999999e-02,  5.9999999e-02,  1.1333334e-01,  2.0000000e-02,
          2.0000000e-02,  2.6666667e-02,  4.6666667e-02,  7.9999998e-02,
          3.9999999e-02,  2.0000000e-02,  4.6666667e-02,  1.3333334e-02,
          5.9999999e-02], dtype=float32),
  0.153618295119

In [20]:
X, y = df.iloc[:,1:-1], df.iloc[:,-1]

,seq_len,molecular_weight,hydrophobicity,instability_index,aromaticity,A %,C %,D %,E %,F %,...,M %,N %,P %,Q %,R %,S %,T %,V %,W %,Y %
0,529,59138.7350,-0.611153,58.945936,0.058601,0.088847,0.024575,0.045369,0.085066,0.035917,...,0.032136,0.041588,0.085066,0.039698,0.069943,0.077505,0.039698,0.064272,0.011342,0.011342
1,170,19301.1649,-0.441765,37.586471,0.064706,0.041176,0.023529,0.094118,0.035294,0.047059,...,0.023529,0.029412,0.052941,0.041176,0.088235,0.064706,0.058824,0.064706,0.005882,0.011765
2,107,12298.1162,-0.919626,46.321495,0.037383,0.074766,0.009346,0.018692,0.065421,0.000000,...,0.009346,0.056075,0.065421,0.065421,0.121495,0.074766,0.037383,0.046729,0.018692,0.018692
3,105,11832.6058,-0.875238,15.507619,0.085714,0.057143,0.019048,0.028571,0.085714,0.038095,...,0.028571,0.047619,0.038095,0.028571,0.019048,0.000000,0.095238,0.028571,0.009524,0.038095
4,318,35554.0730,-0.583019,43.411950,0.088050,0.084906,0.022013,0.059748,0.075472,0.028302,...,0.009434,0.034591,0.066038,0.031447,0.053459,0.062893,0.034591,0.047170,0.022013,0.037736


In [24]:
# balanced random forest 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from statistics import mean
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from imblearn.ensemble import BalancedRandomForestClassifier

#Build SRF model
BRFC = BalancedRandomForestClassifier(n_estimators=150, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
scoring = ('f1', 'recall', 'precision')

#Evaluate BRFC model
scores = cross_validate(BRFC, X, y, scoring=scoring, cv=cv)
print(scores)
#Get average evaluation metrics
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

#Randomly spilt dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
#Train BRFC
BRFC.fit(X_train, y_train)
#BRFC prediction result
y_pred = BRFC.predict(X_test)
#Create confusion matrix
fig = plot_confusion_matrix(BRFC, X_test, y_test, display_labels=['Will Not Buy', 'Will Buy'], cmap='Greens')
plt.title('Balanced Random Forest Confusion Matrix')
plt.show()

/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'BalancedRandomForestClassifier' object has no attribute '_validate_data'

  FitFailedWarning)
/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'BalancedRandomForestClassifier' object has no attribute '_validate_data'

  FitFailedWarning)
/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'Balanced

{'fit_time': array([0.05024695, 0.02645993, 0.02138424, 0.02152014, 0.0212729 ,
       0.02255106, 0.02290726, 0.02630019, 0.02575803, 0.0259099 ,
       0.02140999, 0.02127528, 0.01991606, 0.02032304, 0.01997089,
       0.02111578, 0.02049518, 0.02104974, 0.02157807, 0.02512217,
       0.02251887, 0.02130795, 0.02240205, 0.02173018, 0.02201772,
       0.02161884, 0.02301121, 0.02139902, 0.02402115, 0.02460217]), 'score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'test_f1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan]), 'test_recall': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan]), 'test_precision': array([nan, nan, nan, nan, nan, nan, nan, n

AttributeError: 'BalancedRandomForestClassifier' object has no attribute '_validate_data'